divide and conquer

- transform into a problem we already know how to solve, or that we understand
- to detect duplicates
- first, sort
- then, check adjacent elements

## Recursion

In [2]:
def factorial1(n):
    result = 1
    for i in range(n):
        result *= (i+1)
    return result

In [3]:
def factorial2(n):
    if n == 1:
        return 1
    else:
        return n * factorial2(n-1)

In [4]:
factorial1(0)

1

In [6]:
factorial2(1)

1

In [7]:
def size1(s):
    return 1 + sum(size1(child) for child in s.hyponyms())

In [12]:
# longer and harder to understand
# iterative
# procedural
def size2(s):
    layer = [s]
    total = 0
    while layer:
        total += len(layer)
        layer = [h for c in layer for h in c.hyponyms()]
    return total

In [9]:
from nltk.corpus import wordnet as wn
dog = wn.synset('dog.n.01')

In [10]:
size1(dog)

190

In [13]:
size2(dog)

190

In [14]:
# retrieval
# letter trie

In [15]:
def insert(trie, key, value):
    if key:
        first, rest = key[0], key[1:]
        if first not in trie:
            trie[first] = {}
        insert(trie[first], rest, value)
    else:
        trie['value'] = value

In [16]:
trie = {}
insert(trie, 'chat', 'cat')
insert(trie, 'chien', 'dog')
insert(trie, 'chair', 'flesh')
insert(trie, 'chic', 'stylish')

In [17]:
trie = dict(trie) # for nicer printing

In [18]:
trie['c']['h']['a']['t']['value']

'cat'

In [20]:
import pprint
pprint.pprint(trie, width=40)

{'c': {'h': {'a': {'i': {'r': {'value': 'flesh'}},
                   't': {'value': 'cat'}},
             'i': {'c': {'value': 'stylish'},
                   'e': {'n': {'value': 'dog'}}}}}}


- simplicity
- expensive, each loop -> state info into the stack